<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/19_Feature_Engineering_for_NLP_in_Python/03_N-Gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3. N-Gram models


Learn about n-gram modeling and use it to perform sentiment analysis on movie reviews.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/ted.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
ted = pd.read_csv(url)


display(ted.head())

,transcript,url
0,"We're going to talk — my — a new lecture, just...",https://www.ted.com/talks/al_seckel_says_our_b...
1,"This is a representation of your brain, and yo...",https://www.ted.com/talks/aaron_o_connell_maki...
2,It's a great honor today to share with you The...,https://www.ted.com/talks/carter_emmart_demos_...
3,"My passions are music, technology and making t...",https://www.ted.com/talks/jared_ficklin_new_wa...
4,It used to be that if you wanted to get a comp...,https://www.ted.com/talks/jeremy_howard_the_wo...


In [2]:
!python3 pip install spacy
!python3 -m spacy download en_core_web_sm

python3: can't open file '/content/pip': [Errno 2] No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Load en_core_web_sm and create an nlp object
import spacy
nlp = spacy.load("en_core_web_sm")

### Building a bag of words model

**Word vectors with a given vocabulary**  
You have been given a corpus of documents and you have computed the vocabulary of the corpus to be the following: V: a, an, and, but, can, come, evening, forever, go, i, men, may, on, the, women

Which of the following corresponds to the bag of words vector for the document "men may come and men may go but i go on forever"?

    -> (0, 0, 1, 1, 0, 1, 0, 1, 2, 1, 2, 2, 1, 0, 0)

     Each value in the vector corresponds to the frequency of the corresponding word in the vocabulary.

**BoW model for movie taglines**  
In this exercise, you have been provided with a corpus of more than 7000 movie tag lines. Your job is to generate the bag of words representation bow_matrix for these taglines. For this exercise, we will ignore the text preprocessing step and generate bow_matrix directly.

We will also investigate the shape of the resultant bow_matrix. The first five taglines in corpus have been printed to the console for you to examine.

In [5]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/movie_overviews.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
corpus = pd.read_csv(url)


display(corpus.head())

# Create CountVectorizer object
vectorizer = CountVectorizer()

# Fill NaN values in 'tagline' column with empty strings
corpus['tagline'] = corpus['tagline'].fillna('')

# Generate matrix of word vectors
bow_matrix = vectorizer.fit_transform(corpus["tagline"])

# Print the shape of bow_matrix
print(bow_matrix.shape)

# You now know how to generate a bag of words representation for a given corpus of documents.
# Notice that the word vectors created have more than 6600 dimensions. However,
# most of these dimensions have a value of zero since most words do not occur in a particular tagline.

,id,title,overview,tagline
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",NaN
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...


(9099, 6614)


**Analyzing dimensionality and preprocessing**  
In this exercise, you have been provided with a lem_corpus which contains the pre-processed versions of the movie taglines from the previous exercise. In other words, the taglines have been lowercased and lemmatized, and stopwords have been removed.

Your job is to generate the bag of words representation bow_lem_matrix for these lemmatized taglines and compare its shape with that of bow_matrix obtained in the previous exercise. The first five lemmatized taglines in lem_corpus have been printed to the console for you to examine.



In [6]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_lem_matrix = vectorizer.fit_transform(lem_corpus)

# Print the shape of bow_lem_matrix
print(bow_lem_matrix.shape)

NameError: name 'lem_corpus' is not defined

In [7]:
# The lemmatized corpus was not provided, so I will create a dummy lemmatized corpus for demonstration purposes.
# In a real scenario, you would replace this with your actual lemmatized data.
lem_corpus = corpus['tagline'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if not token.is_stop]))

# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors for the lemmatized corpus
bow_lem_matrix = vectorizer.fit_transform(lem_corpus)

# Print the shape of bow_lem_matrix
print(bow_lem_matrix.shape)

# Compare the shapes
print(f"Shape of original bow_matrix: {bow_matrix.shape}")
print(f"Shape of lemmatized bow_lem_matrix: {bow_lem_matrix.shape}")

#  Notice how the number of features have reduced significantly from around 6600 to around 5223 for pre-processed movie taglines.
#  The reduced number of dimensions on account of text preprocessing usually leads to better performance when conducting machine
#  learning and it is a good idea to consider it. However, as mentioned in a previous lesson, the final decision always depends on the nature of the application.

(9099, 5428)
Shape of original bow_matrix: (9099, 6614)
Shape of lemmatized bow_lem_matrix: (9099, 5428)


**Mapping feature indices with feature names**   
In the lesson video, we had seen that CountVectorizer doesn't necessarily index the vocabulary in alphabetical order. In this exercise, we will learn to map each feature index to its corresponding feature name from the vocabulary.

We will use the same three sentences on lions from the video. The sentences are available in a list named corpus and has already been printed to the console.

In [9]:
# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_matrix = vectorizer.fit_transform(corpus["tagline"])

# Convert bow_matrix into a DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray())

# Map the column names to vocabulary
bow_df.columns = vectorizer.get_feature_names_out()

# Print bow_df
print(bow_df)

#  Observe that the column names refer to the token whose frequency is being recorded.
#  Therefore, since the first column name is an, the first feature represents the number of times the word 'an' occurs in a particular sentence.
#  get_feature_names() essentially gives us a list which represents the mapping of the feature indices to the feature name in the vocabulary.



      000  007  05  06  08  09  10  100  1000  1001  ...  zhivago  zip  \
0       0    0   0   0   0   0   0    0     0     0  ...        0    0   
1       0    0   0   0   0   0   0    0     0     0  ...        0    0   
2       0    0   0   0   0   0   0    0     0     0  ...        0    0   
3       0    0   0   0   0   0   0    0     0     0  ...        0    0   
4       0    0   0   0   0   0   0    0     0     0  ...        0    0   
...   ...  ...  ..  ..  ..  ..  ..  ...   ...   ...  ...      ...  ...   
9094    0    0   0   0   0   0   0    0     0     0  ...        0    0   
9095    0    0   0   0   0   0   0    0     0     0  ...        0    0   
9096    0    0   0   0   0   0   0    0     0     0  ...        0    0   
9097    0    0   0   0   0   0   0    0     0     0  ...        0    0   
9098    0    0   0   0   0   0   0    0     0     0  ...        0    0   

      zombies  zone  zones  zoo  zorba  zwei  ˈfil  ˌrän  
0           0     0      0    0      0     0     0  

### Building a BoW Naive Bayes classifier

**BoW vectors for movie reviews**  
In this exercise, you have been given two pandas Series, X_train and X_test, which consist of movie reviews. They represent the training and the test review data respectively. Your task is to preprocess the reviews and generate BoW vectors for these two sets using CountVectorizer.

Once we have generated the BoW vector matrices X_train_bow and X_test_bow, we will be in a very good position to apply a machine learning model to it and conduct sentiment analysis.



In [16]:
# Import CountVectorizer and train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/movie_reviews_clean.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
review = pd.read_csv(url)


display(review.head())

# If you have a different DataFrame or column for movie reviews, please adjust accordingly.
X = review['review']
y = review['sentiment'] # Assuming 'title' can be used as a dummy target variable for splitting

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)


# Create a CountVectorizer object
vectorizer = CountVectorizer(lowercase=True, stop_words='english')

# Fit and transform X_train
X_train_bow = vectorizer.fit_transform(X_train)

# Transform X_test
X_test_bow = vectorizer.transform(X_test)

# Print shape of X_train_bow and X_test_bow
print(X_train_bow.shape)
print(X_test_bow.shape)

# You now have a good idea of preprocessing text and transforming them into their bag-of-words representation using CountVectorizer.
# In this exercise, you have set the lowercase argument to True. However, note that this is the default value of lowercase and passing it explicitly is not necessary.
# Also, note that both X_train_bow and X_test_bow have 8158 features. There were words present in X_test that were not in X_train.
# CountVectorizer chose to ignore them in order to ensure that the dimensions of both sets remain the same.



,review,sentiment
0,this anime series starts out great interesting...,0
1,some may go for a film like this but i most as...,0
2,i ve seen this piece of perfection during the ...,1
3,this movie is likely the worst movie i ve ever...,0
4,it ll soon be 10 yrs since this movie was rele...,1


(250, 7887)
(750, 7887)


**Predicting the sentiment of a movie review**  
In the previous exercise, you generated the bag-of-words representations for the training and test movie review data. In this exercise, we will use this model to train a Naive Bayes classifier that can detect the sentiment of a movie review and compute its accuracy. Note that since this is a binary classification problem, the model is only capable of classifying a review as either positive (1) or negative (0). It is incapable of detecting neutral reviews.

In case you don't recall, the training and test BoW vectors are available as X_train_bow and X_test_bow respectively. The corresponding labels are available as y_train and y_test respectively. Also, for you reference, the original movie review dataset is available as df.

In [18]:
# Import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

# Create a MultinomialNB object
clf = MultinomialNB()

# Fit the classifier
clf.fit(X_train_bow, y_train)

# Measure the accuracy
accuracy = clf.score(X_test_bow, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
review = "The movie was terrible. The music was underwhelming and the acting mediocre."
prediction = clf.predict(vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))

# You have successfully performed basic sentiment analysis. Note that the accuracy of the classifier is 73.2%. Considering the fact that it was trained on only 750 reviews,
# this is reasonably good performance. The classifier also correctly predicts the sentiment of a mini negative review which we passed into it.

The accuracy of the classifier on the test set is 0.756
The sentiment predicted by the classifier is 0


### Building n-gram models

**n-gram models for movie tag lines**   
In this exercise, we have been provided with a corpus of more than 9000 movie tag lines. Our job is to generate n-gram models up to n equal to 1, n equal to 2 and n equal to 3 for this data and discover the number of features for each model.

We will then compare the number of features generated for each model.

In [20]:
# Generate n-grams upto n=1
vectorizer_ng1 = CountVectorizer(ngram_range=(1,1))
ng1 = vectorizer_ng1.fit_transform(corpus["tagline"])

# Generate n-grams upto n=2
vectorizer_ng2 = CountVectorizer(ngram_range=(1,2))
ng2 = vectorizer_ng2.fit_transform(corpus["tagline"])

# Generate n-grams upto n=3
vectorizer_ng3 = CountVectorizer(ngram_range=(1, 3))
ng3 = vectorizer_ng3.fit_transform(corpus["tagline"])

# Print the number of features for each model
print("ng1, ng2 and ng3 have %i, %i and %i features respectively" % (ng1.shape[1], ng2.shape[1], ng3.shape[1]))

# You now know how to generate n-gram models containing higher order n-grams. Notice that ng2 has over 37,000
# features whereas ng3 has over 76,000 features. This is much greater than the 6,000 dimensions obtained for ng1.
# As the n-gram range increases, so does the number of features, leading to increased computational costs and a problem known as the curse of dimensionality.



ng1, ng2 and ng3 have 6614, 37100 and 76881 features respectively


**Higher order n-grams for sentiment analysis**  
Similar to a previous exercise, we are going to build a classifier that can detect if the review of a particular movie is positive or negative. However, this time, we will use n-grams up to n=2 for the task.

The n-gram training reviews are available as X_train_ng. The corresponding test reviews are available as X_test_ng. Finally, use y_train and y_test to access the training and test sentiment classes respectively.

In [43]:
# Import CountVectorizer and train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/movie_reviews_clean.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
review = pd.read_csv(url)


display(review.head())

# If you have a different DataFrame or column for movie reviews, please adjust accordingly.
X = review['review']
y = review['sentiment'] # Assuming 'title' can be used as a dummy target variable for splitting

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a CountVectorizer object for n-grams up to n=2
ng_vectorizer = CountVectorizer(ngram_range=(1, 2), lowercase=True, stop_words='english')

# Fit and transform X_train
X_train_ng = ng_vectorizer.fit_transform(X_train)

# Transform X_test
X_test_ng = ng_vectorizer.transform(X_test)

# Define an instance of MultinomialNB
clf_ng = MultinomialNB()

# Fit the classifier
clf_ng.fit(X_train_ng, y_train)

# Measure the accuracy
accuracy = clf_ng.score(X_test_ng,y_test )
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
review = "The movie was not good. The plot had several holes and the acting lacked panache."
prediction = clf_ng.predict(ng_vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))

,review,sentiment
0,this anime series starts out great interesting...,0
1,some may go for a film like this but i most as...,0
2,i ve seen this piece of perfection during the ...,1
3,this movie is likely the worst movie i ve ever...,0
4,it ll soon be 10 yrs since this movie was rele...,1


The accuracy of the classifier on the test set is 0.760
The sentiment predicted by the classifier is 0


**Comparing performance of n-gram models**  
You now know how to conduct sentiment analysis by converting text into various n-gram representations and feeding them to a classifier. In this exercise, we will conduct sentiment analysis for the same movie reviews from before using two n-gram models: unigrams and n-grams upto n equal to 3.

We will then compare the performance using three criteria: accuracy of the model on the test set, time taken to execute the program and the number of features created when generating the n-gram representation.

***Initialize a CountVectorizer object such that it generates unigrams.***

In [51]:
import time

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/movie_reviews_clean.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
review = pd.read_csv(url)


start_time = time.time()

display(review.head())

# Splitting the data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(review['review'], review['sentiment'], test_size=0.5, random_state=42, stratify=review['sentiment'])

# Generating ngrams
vectorizer = CountVectorizer(ngram_range=(1, 1), lowercase=True, stop_words='english')
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X, train_y)

# Print accuracy, time and number of dimensions
print("The program took %.3f seconds to complete. The accuracy on the test set is %.2f. The ngram representation had %i features." % (time.time() - start_time, clf.score(test_X, test_y), train_X.shape[1]))

,review,sentiment
0,this anime series starts out great interesting...,0
1,some may go for a film like this but i most as...,0
2,i ve seen this piece of perfection during the ...,1
3,this movie is likely the worst movie i ve ever...,0
4,it ll soon be 10 yrs since this movie was rele...,1


The program took 0.240 seconds to complete. The accuracy on the test set is 0.75. The ngram representation had 12057 features.


***Initialize a CountVectorizer object such that it generates ngrams upto n=3.***

In [50]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/19_Feature_Engineering_for_NLP_in_Python/Dataset/movie_reviews_clean.csv"
# Read the CSV file
# Apply pd.to_numeric only to relevant columns, excluding 'text'
review = pd.read_csv(url)


display(review.head())

start_time = time.time()
# Splitting the data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(review['review'], review['sentiment'], test_size=0.5, random_state=42, stratify=review['sentiment'])

# Generating ngrams
vectorizer = CountVectorizer(ngram_range=(1, 3), lowercase=True, stop_words='english')
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X, train_y)

# Print accuracy, time and number of dimensions
print("The program took %.3f seconds to complete. The accuracy on the test set is %.2f. The ngram representation had %i features." % (time.time() - start_time, clf.score(test_X, test_y), train_X.shape[1]))


# The program took around 0.2 seconds in the case of the unigram model and more than 10 times longer for the higher order n-gram model.
# The unigram model had over 12,000 features whereas the n-gram model for
# upto n=3 had over 178,000! Despite taking higher computation time and generating more features, the classifier only performs marginally
# better in the latter case, producing an accuracy of 77% in comparison to the 75% for the unigram model.

,review,sentiment
0,this anime series starts out great interesting...,0
1,some may go for a film like this but i most as...,0
2,i ve seen this piece of perfection during the ...,1
3,this movie is likely the worst movie i ve ever...,0
4,it ll soon be 10 yrs since this movie was rele...,1


The program took 1.042 seconds to complete. The accuracy on the test set is 0.76. The ngram representation had 112775 features.


----

----

-----

----

## Something AI has been able to achive
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error.

## Import tfidfvectorizer

### Subtask:
Import the necessary class from scikit-learn to generate TF-IDF vectors.


**Reasoning**:
The subtask is to import the `TfidfVectorizer` class. This can be done in a code block.



In [33]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Generate tf-idf vectors

### Subtask:
Create TF-IDF vectors for the training and test sets (`X_train` and `X_test`).


**Reasoning**:
Create TF-IDF vectors for the training and test sets using TfidfVectorizer.



In [34]:
# Create a TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')

# Fit and transform X_train
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform X_test
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Print shape of X_train_tfidf and X_test_tfidf
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(250, 7887)
(750, 7887)


## Train a classifier

### Subtask:
Train a classifier (e.g., Multinomial Naive Bayes) using the TF-IDF vectors.


**Reasoning**:
The subtask requires training a Multinomial Naive Bayes classifier using the TF-IDF vectors. This involves importing the classifier, creating an instance, and then fitting it to the training data.



In [35]:
# Import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

# Create a MultinomialNB object
clf_tfidf = MultinomialNB()

# Fit the classifier
clf_tfidf.fit(X_train_tfidf, y_train)

MultinomialNB()

## Evaluate the classifier

### Subtask:
Evaluate the performance of the classifier on the test set.


**Reasoning**:
Calculate and print the accuracy of the TF-IDF classifier on the test set, then predict and print the sentiment of an example review.



In [36]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
# Measure the accuracy
accuracy_tfidf = clf_tfidf.score(X_test_tfidf, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy_tfidf)

# Predict the sentiment of a negative review
review = "The movie was not good. The plot had several holes and the acting lacked panache."
prediction_tfidf = clf_tfidf.predict(tfidf_vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction_tfidf))

The accuracy of the classifier on the test set is 0.748
The sentiment predicted by the classifier is 0


## Summary:

### Data Analysis Key Findings

* The accuracy of the classifier on the test set is 0.748.
* The classifier predicted the sentiment of a given negative review as 0 (negative).

### Insights or Next Steps

* The model's accuracy of 74.8% suggests it performs reasonably well, but there's room for improvement.
* Consider exploring other classification algorithms or feature engineering techniques to potentially enhance performance.
